In [ ]:
import re
import os
import sys
import nltk
import pickle
import base64
import copy
import time

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1

In [ ]:
fields = {}
fields["t"] = open("index/title.txt","r")
fields["c"] = open("index/category.txt","r")
fields["i"] = open("index/infobox.txt","r")
fields["b"] = open("index/body_text.txt","r")
# print(fields)

In [ ]:
words_position = pickle.load(open("index/word_position.pickle", "rb"))
title_position = pickle.load(open("index/title_doc_no.pickle", "rb"))

In [ ]:
while True:
    query_result = []
    query = input("> ")
    if (query == "quit") :
        break
    start = time.time()
    query = query.lower().strip()
    if ":" in query:
        tmp_result = []
        flag = 0
        query_fields = query.split(",")
        for queries in query_fields:
            field,query = queries.split(":")
            field = field.strip()
            if field == "ref" or field == "ext" or field == "body":
                field = "b"
            elif field == "title":
                field = "t"
            elif field == "category":
                field = "c"
            elif field == "infobox":
                field = "i"
            words = query.split(" ")
            for word in words:
                word = word.strip()
                word = stemmer.stem(word)
                if word and word not in stop_words:
                    if word in words_position and field in words_position[word]:
                        docs = []
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[: -1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            docs.append(doc.split(":")[0])

                        if flag == 0:
                            tmp_result = copy.deepcopy(docs)
                            flag = 1
                        else:
                            tmp_result = copy.deepcopy(list(set(tmp_result) & set(docs)))
            
        for doc_id in tmp_result:
            query_result.append(title_position[int(doc_id)])
    else:
        flag = 0
        tmp_result = []
        query_words = query.split(" ")
        for word in query_words:
            word = word.strip()
            word = stemmer.stem(word)
            if word not in stop_words:
                if word in words_position:
                    docs = []
                    for field in words_position[word].keys():
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[: -1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            docs.append(doc.split(":")[0])
                    if flag == 0:
                        tmp_result = copy.deepcopy(docs)
                        flag = 1
                    else:
                        tmp_result = copy.deepcopy(list(set(tmp_result) & set(docs)))
        
        for doc_id in tmp_result:
            query_result.append(title_position[int(doc_id)])
    
    end = time.time()
    print("Time Taken :- ",(end-start))
    if len(query_result) == 0:
        print("No result found")
    else:
        query_result = set(query_result)
        print("Number of Results :- ",len(query_result))
        for result in query_result:
            print(result)
                        